In [1]:
import pandas as pd

df = pd.read_csv("predicted_nodes.csv")

In [1]:
import pandas as pd

df = pd.read_csv("dataset.csv")

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import os.path

chrome_options = Options()
chrome_options.add_argument("--headless") # Ensure GUI is off
chrome_options.add_argument("--no-sandbox")

homedir = os.path.expanduser("~")
chrome_options.binary_location = f"{homedir}/chrome-linux64/chrome"
webdriver_service = Service(f"{homedir}/chromedriver-linux64/chromedriver")

driver = webdriver.Chrome(service=webdriver_service, options=chrome_options)

In [3]:
import ast
from selenium.webdriver.remote.webelement import WebElement

import re

def keep_assignments(code_snippet):
    # Regex to match variable assignments. This pattern assumes variable names are valid Python identifiers
    # and captures typical assignment statements, excluding those that might appear in comments or strings.
    pattern = r'^\s*[a-zA-Z_][a-zA-Z0-9_]*\s*=\s*.+'

    # Filter and keep only lines with variable assignments
    assignments = [line for line in code_snippet.split('\n') if re.match(pattern, line)]

    # Join the filtered lines back into a string
    return "\n".join(assignments)

# This function will be used to visit each node in the AST
class VariableVisitor(ast.NodeVisitor):
    def __init__(self):
        super().__init__()
        self.output = []
    
    def visit_Assign(self, node):
        
        # For each assignment, print the targets (variables)
        for target in node.targets:
            if isinstance(target, ast.Name):  # Ensure it's a variable assignment
                self.output.append(target.id)
def get_web_elements(code_snippet):
    # Parse the code snippet into an AST
    parsed_code = ast.parse(code_snippet)

    # Create a visitor instance and use it to visit the nodes in the parsed AST
    visitor = VariableVisitor()
    visitor.visit(parsed_code)
    variables = visitor.output

    output = []
    variable_type = WebElement
    for var_name in variables:
            # Use globals() to get the variable from its name since we are in the global scope
        var_value = globals().get(var_name, None)
        
        # Check if the variable exists and its type
        if var_value is not None and isinstance(var_value, variable_type):
            output.append(var_value)
    return output

In [4]:
from tqdm import tqdm

output = []

for index, row in tqdm(df.iterrows()):
    html = row['html']
    file_path = 'sample_page.html'
    html_content = html

    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(html_content)

    code = row['code']
    code = keep_assignments(code)
    
    abs_file_path = os.path.abspath("sample_page.html")

    # Use the file:/// protocol to load the local HTML file
    driver.get(f"file:///{abs_file_path}")
    
    exec(code)
    
    variables = get_web_elements(code)
    
    # We assume there is only one variable defined
    variable = variables[0]
    outer_html = variable.get_attribute("outerHTML")
    
    output.append(outer_html)
    
print(output)

10it [00:05,  1.88it/s]

['<a class="btn-mktg home-campaign-enterprise btn-muted-mktg" data-analytics-event="{&quot;category&quot;:&quot;Start a free enterprise trial&quot;,&quot;action&quot;:&quot;click to Start a free enterprise trial&quot;,&quot;label&quot;:&quot;ref_cta:Start a free enterprise trial;&quot;}" href="/organizations/enterprise_plan?ref_cta=Start+a+free+enterprise+trial&amp;ref_loc=Home+campaign+hero&amp;ref_page=%2F" data-test-selector="start-trial-button">\n  Start a free enterprise trial\n  \n  <svg xmlns="http://www.w3.org/2000/svg" class="octicon arrow-symbol-mktg" width="16" height="16" viewBox="0 0 16 16" fill="none"><path fill="currentColor" d="M7.28033 3.21967C6.98744 2.92678 6.51256 2.92678 6.21967 3.21967C5.92678 3.51256 5.92678 3.98744 6.21967 4.28033L7.28033 3.21967ZM11 8L11.5303 8.53033C11.8232 8.23744 11.8232 7.76256 11.5303 7.46967L11 8ZM6.21967 11.7197C5.92678 12.0126 5.92678 12.4874 6.21967 12.7803C6.51256 13.0732 6.98744 13.0732 7.28033 12.7803L6.21967 11.7197ZM6.21967 4.2803

In [5]:
df["outer_html"] = output

In [6]:
from IPython.display import HTML, display

for html in df['outer_html']:
    display(HTML(html))